In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName("transformation_actions")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/02 06:33:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# 스파크 설정 환경 확인
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.driver.port', '43821'),
 ('spark.app.id', 'local-1685687616602'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.startTime', '1685687614945'),
 ('spark.executor.id', 'driver'),
 ('spark.app.name', 'transformation_actions'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.host', 'ip-172-31-47-237.ap-northeast-3.compute.internal'),
 ('spark.ui.showConsoleProgress', 'true')]

# Python 리스트를 RDD로 만들기

In [4]:
# 패럴라이즈 사용
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:274

1. RDD내의 모든 데이터 가져오기
- collect()

In [5]:
foods.collect()


[Stage 0:===========================================================(1 + 0) / 1]



['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '짬뽕', '짜장면', '짜장면', '짜장면', '라면', '우동', '라면']

2. 각 데이터 별 개수 세기
- countByValue()

In [6]:
foods.countByValue()

defaultdict(int,
            {'짜장면': 4,
             '마라탕': 1,
             '짬뽕': 2,
             '떡볶이': 1,
             '쌀국수': 1,
             '라면': 2,
             '우동': 1})

3. 상위 n개의 데이터 가져오기
- take(n)

In [7]:
foods.take(3)

['짜장면', '마라탕', '짬뽕']

4. 처음 1개의 데이터만 가져오기 
- first()

In [8]:
foods.first()

'짜장면'

5. RDD내의 데이터 개수 세기
- count

In [9]:
foods.count()

12

6. 중복데이터 제거(Transformations)
- distinct

In [10]:
fd = foods.distinct()
fd

PythonRDD[9] at RDD at PythonRDD.scala:53

In [11]:
fd.collect()

['짜장면', '마라탕', '짬뽕', '떡볶이', '쌀국수', '라면', '우동']

In [12]:
# 중복을 제외한 개수는 distinct 후 Count
foods.distinct().count()

7

* 중요한 기능은 아님!
- 요소를 하나씩 꺼내서 함수에 저장할 때 사용
- foreach
- 마스터한테 리턴이 안되고 워커들 내에서만 작동
- 연산의 결과를 로그화 시킬 때 많이 사용

In [13]:
foods.foreach(lambda x: print(x))

짜장면
마라탕
짬뽕
떡볶이
쌀국수
짬뽕
짜장면
짜장면
짜장면
라면
우동
라면


In [14]:
foods.foreach(lambda x: x)

Narrow Transformations
- 1:1 변환을 의미
- 하나의 열을 조작하기 위해 다른 열 및 파티션의 데이터를 사용하지 않는다.
- filter(), map(),flatMap(), sample(), union()

In [15]:
sample_rdd = sc.parallelize([1,2,3])
sample_rdd

ParallelCollectionRDD[17] at readRDDFromFile at PythonRDD.scala:274

map(<task>)
- 데이터를 하나씩 꺼내서 `<func>` 함수가 적용된 새로운 RDD가 만들어 진다.

In [16]:
sample_rdd2 = sample_rdd.map(lambda x : x+2)
sample_rdd2

PythonRDD[18] at RDD at PythonRDD.scala:53

In [17]:
sample_rdd2.collect()

[3, 4, 5]

flatMap(<func>)
- `map` 함수와 거의 비슷하나, `flatMap` 함수는 `map`의 모든 결과를 1차원 배열 형식으로 평평(`flat`)하게 나타낸다.

In [19]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [20]:
moviesRDD = sc.parallelize(movies)
moviesRDD

ParallelCollectionRDD[19] at readRDDFromFile at PythonRDD.scala:274

In [21]:
def split(row):
    row = row.split()
    return row

In [23]:
mapMovies = moviesRDD.map(split)
# mapMovies = moviesRDD.map(lambda x:x.split() )
mapMovies.collect()

[['그린', '북'],
 ['매트릭스'],
 ['토이', '스토리'],
 ['캐스트', '어웨이'],
 ['포드', 'V', '페라리'],
 ['보헤미안', '랩소디'],
 ['빽', '투', '더', '퓨처'],
 ['반지의', '제왕'],
 ['죽은', '시인의', '사회']]

In [24]:
flatMovies = moviesRDD.flatMap(lambda x : x.split())
flatMovies.collect()

['그린',
 '북',
 '매트릭스',
 '토이',
 '스토리',
 '캐스트',
 '어웨이',
 '포드',
 'V',
 '페라리',
 '보헤미안',
 '랩소디',
 '빽',
 '투',
 '더',
 '퓨처',
 '반지의',
 '제왕',
 '죽은',
 '시인의',
 '사회']

# Wide Transformation
- shuffling

In [25]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[22] at readRDDFromFile at PythonRDD.scala:274

`groupBy(<func>)`
- `<func>` : 기준을 마련하는 함수
- 키값을 마련하는 것

In [26]:
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup.collect()

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f210c1a5e20>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f21017962b0>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f2101796310>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f21017963d0>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f2101796400>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f2101796460>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f21017964f0>)]

In [27]:
res =foodsGroup.collect()
res

[('짜', <pyspark.resultiterable.ResultIterable at 0x7f210c1a5bb0>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7f2130294550>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7f2130294520>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7f210c147f40>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7f210c0492b0>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7f2101740ee0>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7f2101796880>)]

In [28]:
for (k,v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
마 ['마라탕']
짬 ['짬뽕', '짬뽕']
떡 ['떡볶이']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']


In [30]:
sc.stop()